In [4]:
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [2]:
import json
from pathlib import Path
from shutil import copyfileobj

import pandas as pd
from requests import Session, HTTPError

In [9]:
template = "https://mtarchive.geol.iastate.edu/%Y/%m/%d/mrms/ncep/ProbSevere/MRMS_PROBSEVERE_%Y%m%d_%H%M00.json"
outdir = Path("/home/leaver2000/sppp/data/new")


def download_archive_data(start: str, end: str, freq: str = "2min", outdir:Path= Path("/home/leaver2000/sppp/archive")) -> None:
    urls = pd.date_range(start=start, end=end,freq=freq).strftime(template)
    # print(urls)
    with Session() as session:
        for url in urls:
            try:
                r = session.get(url, stream=True)
                r.raise_for_status()
                outfile = outdir / r.url.split("/")[-1]

                with outfile.open("w") as f:
                    json.dump(r.json(), f, indent=4)

            except (ConnectionError, HTTPError):
                print("error downloading", url)

if __name__ == "__main__":
    download_archive_data(start="2021-10-11T12:00:00Z", end="2021-11-11T00:00:00Z", outdir=outdir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/leaver2000/sppp/data/new/MRMS_PROBSEVERE_20211011_120000.json'